In [ ]:
#Установим необходимые пакеты Python
!pip3 install psycopg2
!pip3 install psycopg2-binary scikit-learn numpy pymorphy2

In [1]:
# Импортируем библиотеки
import pymorphy2
import re
import psycopg2
import numpy as np

In [2]:
# Подключаемся к PostgreSQL
import psycopg2

try:
    conn = psycopg2.connect(dbname='postgres', user='postgres', password='postgres', host='localhost')
    cursor = conn.cursor()
except:
    # в случае сбоя подключения будет выведено сообщение 
    print('Can`t establish connection to database')

In [3]:
# Настраиваем язык для библиотеки морфологии
import pymorphy3

morph = pymorphy3.MorphAnalyzer(lang='ru')

In [4]:
# объявляем массив кодов ответов и ответов
answer_id=[] 
answer = dict()

In [5]:
# получаем из PostgreSQL список ответов и проиндексируем их.
cursor.execute('SELECT id, answer FROM tab_answer;')
records = cursor.fetchall()
for row in records:
		answer[row[0]]=row[1]  
answer		

{4: '14 дней',
 5: '30 дней',
 6: '5 дней',
 1: '1 день',
 2: '3 дня',
 3: '10 дней',
 7: 'да, я готов поговорить об этом',
 8: 'здравствуйте',
 9: 'скоро'}

In [6]:
# объявляем массив вопросов
questions=[] 

# загрузим вопросы и коды ответов
cursor.execute('SELECT question, answer_id FROM tab_question;')
records = cursor.fetchall()

# посчитаем количество вопросов
transform=0

for row in records:
	phrases=row[0]
	# разбираем вопрос на слова
	words=phrases.split(' ')
	phrase=""
	for word in words:
		# каждое слово из вопроса приводим в нормальную словоформу
		word = morph.parse(word)[0].normal_form 
		# составляем фразу из нормализованных слов
		phrase = phrase + word + " "
		# Если длинна полученной фразы больше 0 добавляем ей в массив вопросов и массив кодов ответов
		if (len(phrase)>0):
			questions.append(phrase.strip())
			answer_id.append(row[1])
			transform=transform+1

# выведем на экран вопросы, ответы и коды ответов
print (questions)
print (answer)
print (answer_id)

# Закроем подключение к PostgreSQL
cursor.close()
conn.close()


['срок', 'срок обработка', 'срок обработка заявка', 'срок', 'срок обработка', 'срок обработка заказ', 'срок', 'срок обработка', 'срок обработка заявка', 'срок', 'срок сдача', 'срок сдача проект', 'срок', 'срок изготовление', 'срок изготовление изделие', 'срок', 'срок возврат', 'срок возврат товар', 'время', 'время доставка', 'я', 'я пора', 'я пора увеличить', 'я пора увеличить зарплата', 'сколько', 'сколько у', 'сколько у я', 'сколько у я отпуск', 'добрый', 'добрый утро', 'новогодний', 'новогодний каникулы']
{4: '14 дней', 5: '30 дней', 6: '5 дней', 1: '1 день', 2: '3 дня', 3: '10 дней', 7: 'да, я готов поговорить об этом', 8: 'здравствуйте', 9: 'скоро'}
[[1], [1], [1], [1], [1], [1], [2], [2], [2], [3], [3], [3], [4], [4], [4], [5], [5], [5], [6], [6], [7], [7], [7], [7], [5], [5], [5], [5], [8], [8], [9], [9]]


In [7]:
import sklearn
# Векторизируем вопросы в огромную матрицу 
# Перемножив фразы на слова из которых они состоят получим числовые значения

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

vectorizer_q = TfidfVectorizer()
vectorizer_q.fit(questions)
matrix_big_q = vectorizer_q.transform(questions)
print ("Размер матрицы: ")
print (matrix_big_q.shape)

Размер матрицы: 
(32, 21)


In [8]:
# Трансформируем матрицу вопросов в меньший размер для уменьшения объема данных
	
svd_q = TruncatedSVD()
svd_q.fit(matrix_big_q)

# получим трансформированную матрицу
matrix_small_q = svd_q.transform(matrix_big_q)

print ("Коэффициент уменьшения матрицы: ")
print ( svd_q.explained_variance_ratio_.sum())

Коэффициент уменьшения матрицы: 
0.2898620302665037


In [9]:
# Тело программы поиска ответов

from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator

def softmax(x):
  #создание вероятностного распределения
  proba = np.exp(-x)
  return proba / sum(proba)

class NeighborSampler(BaseEstimator):
  def __init__(self, k=5, temperature=10.0):
    self.k=k
    self.temperature = temperature
  def fit(self, X, y):
    self.tree_ = BallTree(X)
    self.y_ = np.array(y)
  def predict(self, X, random_state=None):
    distances, indices = self.tree_.query(X, return_distance=True, k=self.k)
    result = []
    for distance, index in zip(distances, indices):
      result.append(np.random.choice(index, p=softmax(distance * self.temperature)))
    return self.y_[result]



In [16]:
from sklearn.pipeline import make_pipeline

ns_q = NeighborSampler()

# answer_id - код ответа в массиве, который получается при поиске ближайшего ответа
ns_q.fit(matrix_small_q, answer_id) 
pipe_q = make_pipeline(vectorizer_q, svd_q, ns_q)


In [ ]:
import telebot

telebot.apihelper.ENABLE_MIDDLEWARE = True

# Укажем token полученный при регистрации бота
bot = telebot.TeleBot("7348375989:AAEjQFcQfe8IbQ8KNgwlVo99ANObmbPJ734")

# Начнем обработку. Если пользователь запустил бота, ответим 
@bot.message_handler(commands=['start'])
def start_message(message):
	bot.send_message(message.from_user.id, " Здравствуйте. Я виртуальный бот training202!")

# Если пользователь что-то написал, ответим
@bot.message_handler(func=lambda message: True)
def get_text_messages(message):
	request=message.text
  
  # разобьём фразу на массив слов, используя split. '\W' - любой символ кроме буквы и цифры
	words= re.split('\W',request)
	phrase=""
	
  # разберем фразу на слова, нормализуем каждое и соберем фразу
	for word in words:
		word = morph.parse(word)[0].normal_form  
		phrase = phrase + word + " "
    # получим код ответа вызывая нашу функцию 
	reply_id    = int(pipe_q.predict([phrase.strip()]))
	# отправим ответ
	bot.send_message(message.from_user.id, answer[reply_id])
	# выведем в консоль вопрос / ответа
	print("Вопрос:", request, " \n\tНормализованный: ", phrase, " \n\t\tОтвет :", answer[reply_id])

# Запустим обработку событий бота
bot.infinity_polling(none_stop=True, interval=1)


C:\Users\Алена\AppData\Local\Temp\ipykernel_21876\3999907491.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  reply_id    = int(pipe_q.predict([phrase.strip()]))


Вопрос: скоро отпуск  
	Нормализованный:  скоро отпуск   
		Ответ : 5 дней
